In [1]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.optim.lr_scheduler import LambdaLR
# from torch.utils.data import DataLoader
# from torchvision import transforms

# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt

# from config import CONFIG
# from model import ResNet18
# from dataset import MNISTLaplacianDataset, MedMNISTDataset

# import tqdm

# from collections import defaultdict

# import importlib.util

# spec = importlib.util.spec_from_file_location("test", "./test.py")
# test_module = importlib.util.module_from_spec(spec)
# spec.loader.exec_module(test_module)
# test = test_module.test  # Use test function/class from test.py

# import warnings
# warnings.filterwarnings("ignore")

# ################
# # Load Dataset #
# ################
# data = np.load(CONFIG['data_path'])
# train_images, train_labels = data['train_images'], data['train_labels']
# test_images, test_labels = data['test_images'], data['test_labels']

# ###################################
# # Train function to compute alpha #
# ###################################
# def lr_lambda(epoch):
#     initial_lr = 0.001  # Initial learning rate
#     if epoch < 50:
#         return initial_lr / initial_lr  # Learning rate remains 0.001
#     elif epoch < 75:
#         return 0.1 * initial_lr / initial_lr  # Delay learning rate to 0.0001 after 50 epochs
#     else:
#         return 0.01 * initial_lr / initial_lr  # Delay learning rate to 0.00001 after 75 epochs
    
# def train(model, train_loader, task, epsilon_p):
#     model.train()
#     total_loss = 0
#     correct = 0
#     total = 0
#     all_targets = []
#     all_outputs = []
    
#     # For tracking alpha values
#     epoch_alpha_values = []
#     batch_alpha_values = []
    
#     if task == "multi-label, binary-class":
#         criterion = nn.BCEWithLogitsLoss()
#     else:
#         criterion = nn.CrossEntropyLoss()
#     criterion.to(CONFIG['device'])

#     lr = 0.001
#     optimizer = optim.Adam(model.parameters(), lr=lr)
#     scheduler = LambdaLR(optimizer, lr_lambda)

#     for epoch in tqdm.tqdm(range(CONFIG['num_epochs'])):
#         epoch_gradients = []
        
#         for inputs, targets in train_loader:
#             inputs, targets = inputs.float().to(CONFIG['device']), targets.to(CONFIG['device'])

#             optimizer.zero_grad()
#             outputs = model(inputs)

#             if task == 'multi-label, binary-class':
#                 targets = targets.to(torch.float32)
#                 loss = criterion(outputs, targets)
                
#                 # Calculate accuracy for multi-label
#                 predicted = (outputs > 0.5).float()
#                 correct += (predicted == targets).all(dim=1).sum().item()
#             else:
#                 targets = targets.squeeze().long()
#                 loss = criterion(outputs, targets)
                
#                 # Calculate accuracy for standard classification
#                 _, predicted = outputs.max(1)
#                 correct += predicted.eq(targets).sum().item()

#             # Compute loss gradient
#             loss.backward()
            
#             # Compute alpha as the maximum L2 norm of gradients per sample
#             batch_gradients = []
#             for i in range(inputs.size(0)):
#                 # Initialize sum of squared gradients for this sample
#                 sample_grad_squared_sum = 0
                
#                 # Accumulate gradients from all parameters
#                 for param in model.parameters():
#                     if param.grad is not None:
#                         # Accumulate squared gradient norm
#                         sample_grad_squared_sum += param.grad.norm(2).item() ** 2
                
#                 # Store the square root (L2 norm)
#                 batch_gradients.append(np.sqrt(sample_grad_squared_sum))
            
#             # Find max gradient norm in this batch
#             if batch_gradients:
#                 batch_max_grad = max(batch_gradients)
#                 batch_alpha_values.append(batch_max_grad)
#                 epoch_gradients.append(batch_max_grad)
            
#             total += targets.size(0)
#             total_loss += loss.item()

#             optimizer.step()

#             # Store targets and outputs for AUC calculation
#             all_targets.extend(targets.cpu().numpy())
#             all_outputs.extend(outputs.detach().cpu().numpy())
        
#         # Find max gradient norm in this epoch
#         if epoch_gradients:
#             epoch_max_grad = max(epoch_gradients)
#             epoch_alpha_values.append((epoch, epoch_max_grad))
        
#         scheduler.step()

#     # Compute final alpha as maximum gradient norm across all batches
#     alpha = max(batch_alpha_values) if batch_alpha_values else 0
    
#     return model, alpha, epoch_alpha_values

# ########
# # Main #
# ########
# # Define transformation
# data_transform = transforms.Compose([transforms.ToTensor()])

# # Initialize dictionaries to store results
# results = defaultdict(lambda: defaultdict(list))
# classwise_results = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
# alpha_results = {}

# # epsilon_values = [0.01, 0.1, 1.0, 10, 100, 1000]
# epsilon_values = [0.1, 1.0, 10]
# num_folds = CONFIG['num_folds']

# # For plotting
# alpha_vs_epsilon = []
# epoch_alpha_data = defaultdict(list)

# for epsilon_p in epsilon_values:
#     print(f"Running for epsilon_p: {epsilon_p}")
            
#     # Create datasets
#     train_dataset = MNISTLaplacianDataset(train_images, train_labels, epsilon_p=epsilon_p, transform=data_transform)
#     test_dataset = MedMNISTDataset(test_images, test_labels, transform=data_transform)

#     # Create data loaders
#     train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True)
#     test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], shuffle=False)

#     # Initialize model
#     model = ResNet18(CONFIG['num_channels'], CONFIG['num_classes']).to(CONFIG['device'])

#     # Train and test the model
#     train_model, alpha, epoch_alpha_values = train(model, train_loader, CONFIG['task'], epsilon_p)
#     acc, auc, class_acc = test(train_model, test_loader, CONFIG['task'])
    
#     # Store results
#     alpha_results[epsilon_p] = alpha
#     alpha_vs_epsilon.append((epsilon_p, alpha))
#     epoch_alpha_data[epsilon_p] = epoch_alpha_values
    
#     results[epsilon_p]['acc'] = acc
#     results[epsilon_p]['auc'] = auc
    
#     for cls, acc_val in class_acc.items():
#         classwise_results[epsilon_p]['acc'][cls] = acc_val

#     # Print results for this epsilon
#     print(f"\nResults for epsilon_p = {epsilon_p}:")
#     print(f"Alpha (gradient bound): {alpha:.4f}")
#     print(f"Accuracy: {acc:.4f}")
#     print(f"AUC: {auc:.4f}")
    
#     del model, train_model, train_dataset, test_dataset, train_loader, test_loader

# # Create and display alpha vs epsilon plot
# plt.figure(figsize=(10, 6))
# eps_values, alpha_values = zip(*alpha_vs_epsilon)
# plt.plot(eps_values, alpha_values, marker='o')
# plt.xscale('log')  # Log scale for epsilon values
# plt.xlabel('Epsilon (ε)')
# plt.ylabel('Alpha (α) - Gradient Bound')
# plt.title('Gradient Bound (α) vs. Noise Parameter (ε)')
# plt.grid(True)
# plt.show()

# # Create and display alpha across epochs plot
# plt.figure(figsize=(12, 8))
# for eps, epoch_data in epoch_alpha_data.items():
#     if epoch_data:
#         epochs, alphas = zip(*epoch_data)
#         plt.plot(epochs, alphas, marker='.', label=f'ε = {eps}')

# plt.xlabel('Epoch')
# plt.ylabel('Alpha (α) - Gradient Bound')
# plt.title('Gradient Bound (α) Across Training Epochs')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Print comprehensive results table
# results_table = pd.DataFrame()
# for eps in epsilon_values:
#     row = {
#         'epsilon_p': eps,
#         'alpha': alpha_results[eps],
#         'accuracy': results[eps]['acc'],
#         'auc': results[eps]['auc']
#     }
#     results_table = pd.concat([results_table, pd.DataFrame([row])], ignore_index=True)

# print("\nComprehensive Results Table:")
# print(results_table)